In [1]:
import torch
import numpy as np
import cv2
import h5py
import pickle

In [2]:
def convert_polar(l):
    '''Cartesian to Polar Co-ordinates. The Radius and Angle is normalized between 0 and 1.
    
    Arguments:
    
    l : np.array
        Normal numpy array with RGB Pixel values
    
    Returns:
    
    hsv : np.array
          Array with polar co-ordinates
    '''
    hsv = np.zeros((l.shape[0], l.shape[1], l.shape[2], 2), dtype=np.float32)
    flow_magnitude, flow_angle = cv2.cartToPolar(l[..., 0].astype(np.float32), l[..., 1].astype(np.float32))
    nans = np.isnan(flow_magnitude)
    if np.any(nans):
        nans = np.where(nans)
        flow_magnitude[nans] = 0.
    hsv[..., 0] = flow_angle/(2.0*np.pi)
    hsv[..., 1] = cv2.normalize(flow_magnitude, None, 0, 1, cv2.NORM_MINMAX)
    return hsv

In [5]:
# Creating the combined numpy array for hdf5 files
size = []
for i in range(40):
    x = np.load(f'./train_data/adl-{i+1:02d}.npy')
    temp = convert_polar(x)
    if i==0:
        l = temp
    else:
        print(l.shape)
        l = np.concatenate([l,temp],axis=0)
    size.append(x.shape[0])

(148, 240, 320, 2)
(326, 240, 320, 2)
(504, 240, 320, 2)
(652, 240, 320, 2)
(830, 240, 320, 2)
(1058, 240, 320, 2)
(1236, 240, 320, 2)
(1414, 240, 320, 2)
(1562, 240, 320, 2)
(1860, 240, 320, 2)
(2158, 240, 320, 2)
(2406, 240, 320, 2)
(2669, 240, 320, 2)
(2902, 240, 320, 2)
(3175, 240, 320, 2)
(3413, 240, 320, 2)
(3641, 240, 320, 2)
(3904, 240, 320, 2)
(4152, 240, 320, 2)
(4420, 240, 320, 2)
(4698, 240, 320, 2)
(4936, 240, 320, 2)
(5154, 240, 320, 2)
(5222, 240, 320, 2)
(5330, 240, 320, 2)
(5423, 240, 320, 2)
(5521, 240, 320, 2)
(5604, 240, 320, 2)
(5727, 240, 320, 2)
(6125, 240, 320, 2)
(6373, 240, 320, 2)
(6571, 240, 320, 2)
(6769, 240, 320, 2)
(6958, 240, 320, 2)
(7236, 240, 320, 2)
(7574, 240, 320, 2)
(7922, 240, 320, 2)
(8265, 240, 320, 2)
(8533, 240, 320, 2)


In [6]:
with h5py.File('adl_dataset.hdf5', 'w') as f:
    dset = f.create_dataset("default", data=l)

In [7]:
index = []
count=0
for s in size:
    for i in range(s-9):
        index.append(count+i)
    count=index[-1]+10

In [8]:
with open("adl_indices.pkl","wb") as f:
    pickle.dump(index, f)

In [10]:
l = None
dset = None
index = None